In [36]:
import networkx as nx
import matplotlib as plt
import numpy as np
import matplotlib.pyplot as pyt

Creamos los graficos que necesitemos

In [41]:
G1 = nx.erdos_renyi_graph(10, 0.4, seed=1) # 10 nodos, probabilidad de 0.4

G2 = nx.barabasi_albert_graph(10, 2, seed=1)

G3 = nx.watts_strogatz_graph(10, 2, 0.2, seed=1)

G4 = nx.random_tree(10, seed=1)

G5 = nx.stochastic_block_model([20, 20], [[1, 0.2], [0.2, 1]], seed=1)


In [18]:
def random_walk(G, start_node,destination_node):
    current_node = start_node
    path = [current_node]
    while current_node != destination_node:
        neighbors = list(G.neighbors(current_node))
        current_node = np.random.choice(neighbors)
        path.append(current_node)
    return path

# Now we are going to create the function hitting time, that calculates the number of steps it takes to go from one node to another using random walks

def hitting_time(G, start_node, destination_node, num_walks=1000):
    hitting_times = []
    # From node_i to node_j
    for _ in range(num_walks):
        path = random_walk(G, start_node, destination_node)
        hitting_times.append(len(path))
    # From node_j to node_i
    for _ in range(num_walks):
        path = random_walk(G, destination_node, start_node)
        hitting_times.append(len(path))
    return np.mean(hitting_times)

In [ ]:
def hit(G):
    M = np.zeros((len(G.nodes), len(G.nodes)))
    for i in range(len(G.nodes)):
        for j in range(i+1, len(G.nodes)):
            M[i, j] = hitting_time(G, i, j)
            M[j, i] = M[i, j]
    return M



def imprimirColores(M):
    pyt.imshow(M)
    pyt.colorbar()
    pyt.title('Matriz de Hitting Time')
    pyt.show()
    
    
imprimirColores(hit(G5))












